In [86]:
import pandas as pd

In [87]:
def time_series_format_preprocessing(df: pd.DataFrame, interval, datetime_col, set_index_flag=False):
    #
    # YS(年初), MS(月初), W(周), D(日), H(小時), T(分鐘), S(秒),
    #
    df = df.copy()
    df[datetime_col] = pd.to_datetime(df[datetime_col], format='%Y/%m/%d %H:%M')#:'%Y-%m-%d %H:%M%S'
    df = df.dropna(subset=[datetime_col])
    # 方法1
    # df = df.set_index(datetime_col)
    # df = df[~df.index.duplicated(keep='first')]
    # df = df.asfreq(interval)
    # return df if set_index_flag else df.reset_index(drop=False)

    # 方法2
    df = df.set_index(datetime_col, append=True)
    def asfreq(df, freq):
        # 防止索引名稱為null
        names = []
        for i, name in enumerate(df.index.names):
            if name is None:
                names.append(f"level_{i}")
            else:
                names.append(name)
        df.index.names = names
        # 重設第一個索引層級
        level_to_reset = df.index.names[0]
        df_reset = df.reset_index(level=level_to_reset)
        df_reset = df_reset[~df_reset.index.duplicated(keep='first')]
        # 重新設置頻率
        df_resampled = df_reset.asfreq(freq)
        df_resampled.reset_index(inplace=True) # 會導致freq設定消失
        # 設置新的索引
        for i, name in enumerate(df.index.names):
            df_resampled.set_index(name, inplace=True, append=True if i > 0 else False)
        return df_resampled

    df = df[~df.index.duplicated(keep='first')]
    df = asfreq(df, interval)

    df.index.levels[1].freq = interval
    return df if set_index_flag else df.reset_index(drop=False, level=df.index.names[1]) # 0是原索引，1是時間索引

In [113]:
data = pd.read_csv("C:/Users/foresight_User/Data/測試資料/Chiller_CH14(01~24).csv").drop(columns=["CONTEXTID"])


In [114]:
data = time_series_format_preprocessing(data,interval="5T",datetime_col="TIMETAG")

C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2768\3522751529.py:30: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df_reset.asfreq(freq)
C:\Users\foresight_User\AppData\Local\Temp\ipykernel_2768\3522751529.py:40: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.index.levels[1].freq = interval


In [115]:
data = data.set_index("TIMETAG")

In [116]:
# some example data
import pandas as pd
from typing import Union
from statsmodels.tsa.vector_ar.var_model import VARResultsWrapper
from statsmodels.tsa.api import VAR

from statsmodels.tsa.base.datetools import dates_from_str
import numpy as np

In [117]:
def cal_maxLag(data:pd.DataFrame):
    n_totobs = len(data)
    ntrend = 1 #len(trend) if trend.startswith("c") else 0
    neqs = data.shape[1]
    max_estimable = (n_totobs - neqs - ntrend) // (1 + neqs)
    if max_estimable > 1:
        return max_estimable
    else:
        return 1

In [118]:

def vectorAutoregression(data:pd.DataFrame,maxlags:Union[int,str]="auto",ic:str=None):
    model = VAR(data)
    # ==== 這邊不要動 =====

    """ 
    這是套件設定的
    trend : str {"n", "c", "ct", "ctt"}
        * "n" - no deterministic terms
        * "c" - constant term
        * "ct" - constant and linear term
        * "ctt" - constant, linear, and quadratic term

    maxlags 不可以超過 max_estimable 的值
    maxlags 為模型擬合最大數值
    statemodel有設定條件，已經寫在下述的程式
    使用者要調整低於 maxlags
    """
    max_estimable = cal_maxLag(data)
    # ==== 這邊不要動 =====

    print(" maxlags 要小於等於: ", max_estimable)

    if maxlags == "auto":
        maxlags = max_estimable
    if maxlags > max_estimable:
        raise Exception(" maxlags 要小於等於: ", max_estimable)


    """
    ic 為評估模型的好壞
    ic = {'aic', 'fpe', 'hqic', 'bic', None}
    Information criterion to use for VAR order selection.
    aic : Akaike
    fpe : Final prediction error
    hqic : Hannan-Quinn
    bic : Bayesian a.k.a. Schwarz
    """

    results = model.fit(maxlags=maxlags, ic=ic)
    print(f"在最大 lag 數目為 {max_estimable} 的情況下，VAR 找出的最佳 lag 為: ",results.k_ar)
    return results

In [119]:
def vectorAutoregressionRelationship(results:VARResultsWrapper,target:str,pvalue_threshold:float=0.05):
    # target  客人關心的 Y 是甚麼，Y 會包含在 results 中

    coef_df = results.params[target]
    pvalues_df = results.pvalues[target]

    # 合并系数和p值
    summary = pd.concat([coef_df, pvalues_df], axis=1)
    summary.columns = ['coef', 'pvalue']
    summary = summary.drop(index="const").reset_index()
    summary_index = summary["index"].str.split(".", expand=True).rename(columns={0:"time_lag",1:"feature"})
    if summary.empty:
        return {"info":"there is no results found from VAR"}
    else:
        summary_index["time_lag"] = summary_index["time_lag"].str.replace("L","").astype(int)
        summary = pd.concat([summary_index,summary],axis=1).drop(columns="index")
        summary = summary[summary["pvalue"]<pvalue_threshold].reset_index(drop=True)
        return summary

In [123]:

def remove_collinearity(data,remain:list=None): #condition
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    # remain 保留Y值
    if data.shape[1] <2:
        return pd.DataFrame()
    
    if (remain != None) and data.shape[1] > 2 :
        remained_data = data.loc[:,remain]
        #data = data.drop(columns=remain)
    #data = data.fillna(method='ffill').fillna(method='bfill')
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.ffill().bfill() 
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]


    # 去除VIF為NAN或INF
    vif_data = vif_data.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    data = data[vif_data["feature"].to_list()]

    # 移除高相關的 feature 
    # 保留低相關的 feature
    
    from itertools import combinations
    relevance_threshold = 0.3
    coef_dataframe = data.corr()
    remain_col_index = ()
    for i_1,i_2 in list(combinations(coef_dataframe,2)):
        coef = coef_dataframe.loc[i_1,i_2]
        if abs(coef) < abs(relevance_threshold):
            remain_col_index += (i_1,i_2)
    feature_index = list(set(remain_col_index))  


    # low_corr_vars = np.where(np.abs(correlation_matrix) < coefficient_threshold)
    # #print(correlation_matrix)
    # low_corr_vars = [(correlation_matrix.index[x], correlation_matrix.columns[y]) for x, y in zip(*low_corr_vars) if x != y and x > y]

    # Remove one of each highly correlated pair
    # if condition == "intersection":
    #     sets =  [set(pair) for pair in low_corr_vars]
    #     intersection = set.intersection(*sets)
    #     remaining_list = list(intersection)
    # if condition == "union":
    #unique_elements = set(element for pair in low_corr_vars for element in pair)
    #remaining_list = list(unique_elements)
    
    #data = data[remaining_list]
    data = data[feature_index]
    if remain != None:
        remain_list = list(set(remain) - set(feature_index))
        if (data.shape[1] > 2) and (len(remain_list)>0):
            remained_data = remained_data.loc[:,remain_list]
            data = pd.concat([data,remained_data],axis=1)
    # impute nan
    #data = data.fillna(method='ffill').fillna(method='bfill')
    return data

In [126]:
isinstance(results,VARResultsWrapper)

True

In [130]:
isinstance(pd.DataFrame([[1,2],[1,2]]),VARResultsWrapper)

False

In [129]:
pd.DataFrame([[1,2],[1,2]])

,0,1
0,1,2
1,1,2


In [121]:
target = "METROLOGY-CH14_01_KW"
clean_data = remove_collinearity(data=data,remain=[target])
if clean_data.shape[1] >= 2:
    maxlags = cal_maxLag(clean_data) # USER 不能指定超過這個的LAG，LAG值也要大於0
    maxlags = 2
    results = vectorAutoregression(clean_data,maxlags=maxlags,ic=None)
    pvalue_threshold = 0.05
    VAR_relationship = vectorAutoregressionRelationship(results=results,target=target,pvalue_threshold=pvalue_threshold)
else:
    print("there is no significant time lag")

 maxlags 要小於等於:  627
在最大 lag 數目為 627 的情況下，VAR 找出的最佳 lag 為:  2


c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 5min will be used.
  self._init_dates(dates, freq)


In [122]:
results

In [97]:

def data_shift_transform(data,var_result,remain:list=None,remove:list=None):
    if remove !=None:
        var_result = var_result[~var_result["feature"].isin(remove)]
    print(var_result)
    for i in range(var_result.shape[0]):
        
        feature = var_result.iloc[i]["feature"]
        time_lag = var_result.iloc[i]["time_lag"]
        data[feature+f"_lag_{time_lag}"] = data[feature].shift(time_lag)
        print(feature)
    data.dropna(inplace=True)
    columns_list = var_result["feature"].to_list()
    
    if remain != None:
        print(columns_list)
        print(remain)
        columns_list = set(columns_list) - set(remain)
        columns_list = list(columns_list)
        print(columns_list)
    data = data.drop(columns=columns_list)
    return data.reset_index()

In [99]:
shifted_data= data_shift_transform(data=data,var_result=VAR_relationship,remain=[target],remove=[target])#.columns

   time_lag                feature      coef    pvalue
0         1  EQ-CH14_01_Evaporator -9.084752  0.000201
1         1      EQ-CH14_CWP_01_hz  1.281838  0.001583
3         1   EQ-CH14_01_Condenser  9.113320  0.000189
4         1      EQ-CH14_CHP_01_hz  2.689287  0.046813
5         2  EQ-CH14_01_Evaporator  5.278220  0.030709
6         2   EQ-CH14_01_Condenser -5.174817  0.033929
EQ-CH14_01_Evaporator
EQ-CH14_CWP_01_hz
EQ-CH14_01_Condenser
EQ-CH14_CHP_01_hz
EQ-CH14_01_Evaporator
EQ-CH14_01_Condenser
['EQ-CH14_01_Evaporator', 'EQ-CH14_CWP_01_hz', 'EQ-CH14_01_Condenser', 'EQ-CH14_CHP_01_hz', 'EQ-CH14_01_Evaporator', 'EQ-CH14_01_Condenser']
['METROLOGY-CH14_01_KW']
['EQ-CH14_01_Condenser', 'EQ-CH14_01_Evaporator', 'EQ-CH14_CWP_01_hz', 'EQ-CH14_CHP_01_hz']


In [104]:
shifted_data.columns

Index(['TIMETAG', 'EQ-CH14_01_CoolingWater_OutTemp',
       'EQ-CH14_01_CoolingWater_InTemp', 'EQ-CH14_01_CoolingWater_TempDiff',
       'EQ-CH14_01_ColdWater_OutTemp', 'EQ-CH14_01_ColdWater_InTemp',
       'EQ-CH14_01_ColdWater_TempDiff', 'EQ-CH14_CHP_01_flow', 'EQ-CH14_01_RT',
       'METROLOGY-CH14_01_KW', 'EQ-CH14_01_Evaporator_lag_1',
       'EQ-CH14_CWP_01_hz_lag_1', 'EQ-CH14_01_Condenser_lag_1',
       'EQ-CH14_CHP_01_hz_lag_1', 'EQ-CH14_01_Evaporator_lag_2',
       'EQ-CH14_01_Condenser_lag_2'],
      dtype='object')

In [105]:
shifted_data.to_csv("C:/Users/foresight_User/Data/測試資料/Chiller_CH14(01~24)_shifted.csv",sep=",",index=False)

In [32]:
#shifted_data.to_excel("C:/Users/foresight_User/Data/測試資料/Chiller_CH14(01~24)_shifted.xlsx")

In [109]:
data = pd.read_csv("C:/Users/foresight_User/Data/測試資料/Chiller_CH14(25~30).csv")#.drop(columns=["CONTEXTID"])
#data = time_series_format_preprocessing(data,interval="5T",datetime_col="TIMETAG")

In [112]:
data.shape

(1772, 15)

In [111]:
data.dropna()

,CONTEXTID,TIMETAG,EQ-CH14_01_CoolingWater_OutTemp,EQ-CH14_01_CoolingWater_InTemp,EQ-CH14_01_CoolingWater_TempDiff,EQ-CH14_01_ColdWater_OutTemp,EQ-CH14_01_ColdWater_InTemp,EQ-CH14_01_ColdWater_TempDiff,EQ-CH14_01_Evaporator,EQ-CH14_01_Condenser,EQ-CH14_CHP_01_hz,EQ-CH14_CHP_01_flow,EQ-CH14_01_RT,EQ-CH14_CWP_01_hz,METROLOGY-CH14_01_KW
0,131354,2022/8/25 00:05,34.9,27.7,7.2,14.1,20.5,6.400000,1.4,-0.8,41.72361,1385.717,2932.735,47.64930,1114.316
1,131355,2022/8/25 00:10,34.8,27.6,7.2,14.1,20.5,6.400000,1.4,-0.8,41.87986,1392.916,2953.031,48.41319,1108.203
2,131356,2022/8/25 00:15,34.7,27.5,7.2,13.9,20.5,6.600000,1.4,-0.8,41.94931,1397.694,3058.322,47.94444,1105.182
3,131357,2022/8/25 00:20,34.7,27.4,7.3,13.9,20.5,6.600000,1.5,-0.8,42.14028,1404.832,3066.101,48.91667,1113.172
4,131358,2022/8/25 00:25,34.6,27.6,7.0,14.0,20.5,6.500000,1.4,-0.8,42.50486,1415.470,3039.979,48.51736,1110.525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,133345,2022/8/31 23:35,35.4,28.8,6.6,13.9,20.6,6.700001,1.6,-0.7,42.17500,1408.387,3109.927,52.16319,1163.866
1768,133346,2022/8/31 23:40,35.4,28.8,6.6,13.9,20.6,6.700001,1.6,-0.6,42.34861,1413.113,3123.055,51.81597,1158.947
1769,133347,2022/8/31 23:45,35.8,29.1,6.7,14.0,20.6,6.600000,1.5,-0.7,42.33125,1409.570,3071.276,52.42361,1172.091
1770,133348,2022/8/31 23:50,35.7,29.1,6.6,14.0,20.6,6.600000,1.6,-0.6,42.33125,1415.470,3076.442,52.33680,1168.835


In [107]:

shifted_data= data_shift_transform(data=data,var_result=VAR_relationship,remain=[target])#.columns
shifted_data.to_csv("C:/Users/foresight_User/Data/測試資料/Chiller_CH14(25~30)_shifted.csv",sep=",",index=False)

   time_lag                feature      coef    pvalue
0         1  EQ-CH14_01_Evaporator -9.084752  0.000201
1         1      EQ-CH14_CWP_01_hz  1.281838  0.001583
2         1   METROLOGY-CH14_01_KW  0.945639  0.000000
3         1   EQ-CH14_01_Condenser  9.113320  0.000189
4         1      EQ-CH14_CHP_01_hz  2.689287  0.046813
5         2  EQ-CH14_01_Evaporator  5.278220  0.030709
6         2   EQ-CH14_01_Condenser -5.174817  0.033929
EQ-CH14_01_Evaporator
EQ-CH14_CWP_01_hz
METROLOGY-CH14_01_KW
EQ-CH14_01_Condenser
EQ-CH14_CHP_01_hz
EQ-CH14_01_Evaporator
EQ-CH14_01_Condenser
['EQ-CH14_01_Evaporator', 'EQ-CH14_CWP_01_hz', 'METROLOGY-CH14_01_KW', 'EQ-CH14_01_Condenser', 'EQ-CH14_CHP_01_hz', 'EQ-CH14_01_Evaporator', 'EQ-CH14_01_Condenser']
['METROLOGY-CH14_01_KW']
['EQ-CH14_CHP_01_hz', 'EQ-CH14_01_Condenser', 'EQ-CH14_01_Evaporator', 'EQ-CH14_CWP_01_hz']


In [41]:

from sktime.datasets import load_airline
from sktime.transformations.series.lag import Lag
X = load_airline()

In [45]:
t = Lag(3)
Xt = t.fit_transform(X)

In [46]:
Xt

Period
1949-01      NaN
1949-02      NaN
1949-03      NaN
1949-04    112.0
1949-05    118.0
           ...  
1960-11    606.0
1960-12    508.0
1961-01    461.0
1961-02    390.0
1961-03    432.0
Freq: M, Name: Number of airline passengers, Length: 147, dtype: float64

In [44]:
X

Period
1949-01    112.0
1949-02    118.0
1949-03    132.0
1949-04    129.0
1949-05    121.0
           ...  
1960-08    606.0
1960-09    508.0
1960-10    461.0
1960-11    390.0
1960-12    432.0
Freq: M, Name: Number of airline passengers, Length: 144, dtype: float64

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.datasets import fetch_dataset
from sklearn.inspection import PartialDependenceDisplay
# import shap
# shap.initjs()

In [14]:
data = fetch_dataset(name="vic_electricity")

vic_electricity
---------------
Half-hourly electricity demand for Victoria, Australia
O'Hara-Wild M, Hyndman R, Wang E, Godahewa R (2022).tsibbledata: Diverse
Datasets for 'tsibble'. https://tsibbledata.tidyverts.org/,
https://github.com/tidyverts/tsibbledata/.
https://tsibbledata.tidyverts.org/reference/vic_elec.html
Shape of the dataset: (52608, 4)


In [30]:
# Create a recursive multi-step forecaster (ForecasterAutoreg)
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123),
                 lags      = 2
             )

forecaster.fit(
    y    = data['Demand'],
    #exog = data['Temperature']
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 52606, number of used features: 2
[LightGBM] [Info] Start training from score 4665.445840


In [31]:
# Training matrices used by the forecaster to fit the internal regressor
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(
                       y    = data['Demand'],
                       #exog = data['Temperature']
                   )

In [38]:
data["Demand"].shift(1).head(10)

Time
2011-12-31 13:00:00            NaN
2011-12-31 13:30:00    4382.825174
2011-12-31 14:00:00    4263.365526
2011-12-31 14:30:00    4048.966046
2011-12-31 15:00:00    3877.563330
2011-12-31 15:30:00    4036.229746
2011-12-31 16:00:00    3865.597244
2011-12-31 16:30:00    3694.097664
2011-12-31 17:00:00    3561.623686
2011-12-31 17:30:00    3433.035352
Freq: 30min, Name: Demand, dtype: float64

In [33]:
pd.concat([X_train,data],axis=1).head(50)

,lag_1,lag_2,Demand,Temperature,Date,Holiday
Time,,,,,,
2011-12-31 13:00:00,NaN,NaN,4382.825174,21.40,2012-01-01,True
2011-12-31 13:30:00,NaN,NaN,4263.365526,21.05,2012-01-01,True
2011-12-31 14:00:00,4263.365526,4382.825174,4048.966046,20.70,2012-01-01,True
2011-12-31 14:30:00,4048.966046,4263.365526,3877.563330,20.55,2012-01-01,True
2011-12-31 15:00:00,3877.563330,4048.966046,4036.229746,20.40,2012-01-01,True
2011-12-31 15:30:00,4036.229746,3877.563330,3865.597244,20.25,2012-01-01,True
2011-12-31 16:00:00,3865.597244,4036.229746,3694.097664,20.10,2012-01-01,True
2011-12-31 16:30:00,3694.097664,3865.597244,3561.623686,19.60,2012-01-01,True
2011-12-31 17:00:00,3561.623686,3694.097664,3433.035352,19.10,2012-01-01,True


In [12]:
display(y_train.head(3))


Time
2011-12-31 14:00:00    4048.966046
2011-12-31 14:30:00    3877.563330
2011-12-31 15:00:00    4036.229746
Freq: 30min, Name: y, dtype: float64